In [184]:
import pandas as pd
import numpy as np
import cv2
import os
import re
import ast

from PIL import Image,ImageDraw

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import torch
import torchvision

from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

from torch.utils.data import DataLoader, Dataset
from torch.utils.data.sampler import SequentialSampler

from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split

In [185]:
# Function to convert a string into a Python object
def f(x):
    return ast.literal_eval(x.rstrip('\r\n'))

# Function to compute the bounding box of a given geometry
def getBounds(geometry):
    try:
        arr = np.array(geometry).T
        xmin = np.min(arr[0])
        ymin = np.min(arr[1])
        xmax = np.max(arr[0])
        ymax = np.max(arr[1])
        return (xmin, ymin, xmax, ymax)
    except:
        return np.nan # Need to return a value in case of error

# Functions to compute the areas of the bounding box
def getWidth(bounds):
    try:
        (xmin, ymin, xmax, ymax) = bounds
        return np.abs(xmax - xmin)
    except:
        return np.nan


def getHeight(bounds):
    try:
        (xmin, ymin, xmax, ymax) = bounds
        return np.abs(ymax - ymin)
    except:
        return np.nan # Need to return a value in case of error

# Functions to extract the absolute values of the x and y coordinates of the bounding box
def getX(bounds):
    try:
        (xmin, ymin, xmax, ymax) = bounds
        return np.abs(xmin)
    except:
        return np.nan

def getY(bounds):
    try:
        (xmin, ymin, xmax, ymax) = bounds
        return np.abs(ymin)
    except:
        return np.nan # Need to return a value in case of error


class Averager:
    """
   To be used for tracking loss and accuracy over multiple iterations of the training loop.

    Attributes:
        current_total (float): The cumulative sum of all values added so far.
        iterations (float): The number of values added so far.
    """

    def __init__(self):
        """
        Initializes the Averager with a total of 0 and no iterations.
        """
        self.current_total = 0.0  # Initialize the total sum to 0.
        self.iterations = 0.0    # Initialize the iteration count to 0.

    def send(self, value):
        """
        Adds a new value to the running total and increments the iteration count.

        Args:
            value (float): The new value to include in the average.
        """
        self.current_total += value  # Add the value to the cumulative total.
        self.iterations += 1         # Increment the iteration count.

    @property
    def value(self):
        """
        Calculates and returns the current average.

        Returns:
            float: The current average of all values sent so far.
                   Returns 0 if no values have been added.
        """
        if self.iterations == 0:  # Avoid division by zero.
            return 0
        else:
            return 1.0 * self.current_total / self.iterations  # Calculate the average.

    def reset(self):
        """
        Resets the Averager, clearing the total and iteration count.
        """
        self.current_total = 0.0  # Reset the total sum to 0.
        self.iterations = 0.0    # Reset the iteration count to 0.

### Using a Neural Network to detect aircraft in satellite images of an airport

The objective here is to train a neural network to detect aircraft in satellite images of an airport, splitting the dataset into training, validation and test
sets.

Information about the airplanes in the images has been provided via an annotations spreadsheet which contain bounding boxes for regions of interest, class of airplane and id for each respective image. This can be extracted and used for validation of the aircrafts detected by the neural network.

In [186]:
HOME = "/Users/louie/Documents/Visual Studio Code/ACT Work/Airbus-Aircraft-Detection"
directory_path = os.path.join(HOME, 'airbus-aircrafts-sample-dataset')
image_path = os.path.join(directory_path, 'images')
annotations_path = os.path.join(directory_path, 'annotations.csv')
annotations = pd.read_csv(annotations_path)
annotations.head()

,id,image_id,geometry,class
0,1,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(135, 522), (245, 522), (245, 600), (135, 600...",Airplane
1,2,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1025, 284), (1125, 284), (1125, 384), (1025,...",Airplane
2,3,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1058, 1503), (1130, 1503), (1130, 1568), (10...",Airplane
3,4,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(813, 1518), (885, 1518), (885, 1604), (813, ...",Airplane
4,5,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(594, 938), (657, 938), (657, 1012), (594, 10...",Airplane


In [187]:
df = pd.read_csv(annotations_path, converters={'geometry': f, 'class': lambda o: 'Aircraft'})

# Create bounds, width and height
df.loc[:,'bounds'] = df.loc[:,'geometry'].apply(getBounds)
df.loc[:,'w'] = df.loc[:,'bounds'].apply(getWidth)
df.loc[:,'h'] = df.loc[:,'bounds'].apply(getHeight)
df.loc[:,'x'] = df.loc[:,'bounds'].apply(getX)
df.loc[:,'y'] = df.loc[:,'bounds'].apply(getY)
df.head(10)

,id,image_id,geometry,class,bounds,w,h,x,y
0,1,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(135, 522), (245, 522), (245, 600), (135, 600...",Aircraft,"(135, 522, 245, 600)",110,78,135,522
1,2,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1025, 284), (1125, 284), (1125, 384), (1025,...",Aircraft,"(1025, 284, 1125, 384)",100,100,1025,284
2,3,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1058, 1503), (1130, 1503), (1130, 1568), (10...",Aircraft,"(1058, 1503, 1130, 1568)",72,65,1058,1503
3,4,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(813, 1518), (885, 1518), (885, 1604), (813, ...",Aircraft,"(813, 1518, 885, 1604)",72,86,813,1518
4,5,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(594, 938), (657, 938), (657, 1012), (594, 10...",Aircraft,"(594, 938, 657, 1012)",63,74,594,938
5,6,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(451, 725), (524, 725), (524, 798), (451, 798...",Aircraft,"(451, 725, 524, 798)",73,73,451,725
6,7,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1543, 1437), (1614, 1437), (1614, 1497), (15...",Aircraft,"(1543, 1437, 1614, 1497)",71,60,1543,1437
7,8,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1485, 1370), (1561, 1370), (1561, 1437), (14...",Aircraft,"(1485, 1370, 1561, 1437)",76,67,1485,1370
8,9,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1968, 1624), (2053, 1624), (2053, 1687), (19...",Aircraft,"(1968, 1624, 2053, 1687)",85,63,1968,1624
9,10,4f833867-273e-4d73-8bc3-cb2d9ceb54ef.jpg,"[(1659, 1674), (1733, 1674), (1733, 1746), (16...",Aircraft,"(1659, 1674, 1733, 1746)",74,72,1659,1674


But first, these annotations and the dataset must be converted into a form that PyTorch can interpret, this will be done using `torch.utils.data` to create a custom class that stores the image ids and the image directory and the bounding box coordinates in the correct format. Once the images have been located, the image data is converted to a tensor and returned with its corresponding label in a tuple.

In [188]:
class AircraftDataset(Dataset):
    def __init__(self, dataframe, image_dir, transforms=None):
        super().__init__()

        # Initializes the custom Dataset to store the unique image IDs, DataFrame, image directory, and optionally any image transformations.
        self.image_ids = dataframe['image_id'].unique()  
        self.df = dataframe  
        self.image_dir = image_dir  
        self.transforms = transforms  # Optional data augmentation or preprocessing, currently set to None.
    
    def __len__(self) -> int:
        # returns the number of unique images in the dataset
        return self.image_ids.shape[0]  

    def __getitem__(self, idx: int):
        # Loads the image at the given index, converts it to a tensor, reads any annotations and labels
        # and returns them as a tuple

        # Get the image ID for the given index.
        image_id = self.image_ids[idx]

        # Filter the DataFrame for records corresponding to this image ID.
        records = self.df[self.df['image_id'] == image_id]

        # Load the image from the image directory using OpenCV.
        image = cv2.imread(f'{self.image_dir}/{image_id}', cv2.IMREAD_COLOR)

        # Convert the image from BGR to RGB and normalize pixel values to [0, 1].
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0  # Normalize pixel values to the range [0, 1].

        # Extract bounding box coordinates from the records.
        boxes = records[['x', 'y', 'w', 'h']].values
        # Convert from [x, y, w, h] format to [x_min, y_min, x_max, y_max].
        boxes[:, 2] = boxes[:, 0] + boxes[:, 2]  # x_max = x_min + width
        boxes[:, 3] = boxes[:, 1] + boxes[:, 3]  # y_max = y_min + height

        # Compute the area of each bounding box.
        area = (boxes[:, 3] - boxes[:, 1]) * (boxes[:, 2] - boxes[:, 0])
        area = torch.as_tensor(area, dtype=torch.float32)  # Convert to PyTorch tensor.

        # Assign a label of 1 to all bounding boxes (single-class dataset).
        labels = torch.ones((records.shape[0],), dtype=torch.int64).numpy()

        # Assume that all instances are not part of a crowd (set `iscrowd` to 0).
        iscrowd = torch.zeros((records.shape[0],), dtype=torch.int64)

        # Create the target dictionary with required metadata.
        target = {
            'boxes': boxes,  # Bounding boxes.
            'labels': labels,  # Object labels.
            'image_id': torch.tensor([idx]),  # Image index as tensor.
            'area': area,  # Area of each bounding box.
            'iscrowd': iscrowd  # Crowd annotations.
        }

        # If transformations are specified, apply them to the image and bounding boxes.
        if self.transforms:
            # Prepare a sample dictionary for the transformations.
            sample = {
                'image': image,  # Input image.
                'bboxes': target['boxes'],  # Bounding boxes.
                'labels': labels  # Labels for the bounding boxes.
            }
            # Apply transformations.
            sample = self.transforms(**sample)
            # Update the image and bounding boxes after transformation.
            image = sample['image']
            target['boxes'] = torch.stack(tuple(map(torch.tensor, zip(*sample['bboxes'])))).permute(1, 0)

        # Return the processed image, target dictionary, and image ID.
        return image, target, image_id

In [189]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(Weights=None)
#model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_320_fpn(pretrained=True)
#model = torchvision.models.detection.fasterrcnn_mobilenet_v3_large_fpn(pretrained=True)

In [190]:
num_classes = 2  # 2 total class: aircraft and background

# get number of input features for the classifier
in_features = model.roi_heads.box_predictor.cls_score.in_features

# replace the pre-trained head with a new one
model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

### Spliting the dataset into training and validation sets

Neural networks require both a training dataset for pattern recognition and a validation dataset to verify the learnt patterns are correct, this means splitting our dataset appropiately. An 80/20 training/validation split is usually used as a rule of thumb for neural networks as large datasets are typically required for effective pattern recognition, but these patterns still need to be evaulated using accurate data. 

Using a smaller split such as a 50/50 split can result in underfitting leading to insuffcient representation of the data distribution, especially in complex neural networks. Using a larger split such as a 90/10 split can result in overfitting which leads to unreliable evaluation metrics due to the validation set misrepresenting the full distribution, however this is sometimes required on small datasets. 

Taking into account the following factors, we will use a 90/10 split due to the size of the dataset being only 103 images.

In [191]:
unique_img = df.image_id.unique() # Get unique image ids from annotations spreadsheet
train_inds, val_inds = train_test_split(range(unique_img.shape[0]), test_size= 0.1) # 90% train, 10% val

train_df = df.iloc[train_inds] # Get training data
valid_df = df.iloc[val_inds] # Get validation data

In [192]:
def collate_fn(batch): # Function to collate data samples into a batch
    return tuple(zip(*batch))

train_dataset = AircraftDataset(train_df, image_path, get_train_transform())
valid_dataset = AircraftDataset(valid_df, image_path, get_valid_transform())

In [ ]:
# split the dataset randomly into a train set and a test set removing any potential bias
indices = torch.randperm(len(train_dataset)).tolist()

In [ ]:
# define training and validation data loaders
train_data_loader = DataLoader(
    train_dataset,
    batch_size=16, # how many samples per batch to load
    shuffle=False, # disabling shuffling of the data, setting to True will randomize the data for each epoch, which prevents overfitting to the order of the data
                   # setting to False ensures a deterministic order of the data, which is good for valiadation and testing runs
    collate_fn=collate_fn # calling collate function
)

valid_data_loader = DataLoader(
    valid_dataset,
    batch_size=8,
    shuffle=False,
    collate_fn=collate_fn
)